In [1]:
import numpy as np
import scipy as sp
import csi
from hcsi import *

In [2]:
inp = csi.loadData('testdata/Demo_DREAM.csv')

In [3]:
np.random.seed(1)

In [4]:
	hnrv = RandomVariableHyperNetwork(inp,'Gene1',2)
	crv = []
	conditions = np.unique([x[0] for x in inp.columns.values])
	for cond in conditions:
		crv.append(RandomVariableCondition(inp,cond,'Gene1',[10,0.1],[1,1],2,True))

In [5]:
hnrv.currentValue

(['Gene3', 'Gene5'], 'Gene1')

In [10]:
crv[4].currentValue

(['Gene6', 'Gene7'], 'Gene1')

In [15]:
np.log(crv[1].em.hypers)

array([-1.6806106 , -2.38229308, -1.06258689])

In [14]:
for i in np.arange(5):
    wll = crv[i].em.hypers[0]
    crv[i].em.hypers[0] = crv[i].em.hypers[1]
    crv[i].em.hypers[1] = wll

In [19]:
kek2 = crv[0].getConditionalDistribution(hnrv.currentValue)

In [20]:
kek2

array([ 0.0238334 ,  0.02156489,  0.02633945,  0.02156506,  0.02633997,
        0.02156494,  0.02156467,  0.02156494,  0.02156471,  0.02156549,
        0.02383262,  0.01951253,  0.02383286,  0.01951258,  0.01951235,
        0.01951257,  0.01951237,  0.01951278,  0.02383269,  0.02910959,
        0.02383272,  0.02383246,  0.02383272,  0.02383248,  0.02383302,
        0.02383304,  0.0195126 ,  0.01951241,  0.0195126 ,  0.01951243,
        0.01951295,  0.0238329 ,  0.02383265,  0.0238329 ,  0.02383266,
        0.02383336,  0.01951245,  0.01951265,  0.01951247,  0.01951282,
        0.01951242,  0.01951238,  0.01951259,  0.01951243,  0.01951282,
        0.0195126 ])

In [24]:
np.random.seed(1)
valInd = roulette(kek2)

In [25]:
valInd

19

In [26]:
np.random.seed(1)

In [27]:
	hnrv = RandomVariableHyperNetwork(inp,'Gene1',2)
	crv = []
	conditions = np.unique([x[0] for x in inp.columns.values])
	for cond in conditions:
		crv.append(RandomVariableCondition(inp,cond,'Gene1',[10,0.1],[1,1],2,True))
	gibbs = GibbsHCSI(crv,hnrv)

In [28]:
gibbs.rvHyperNetwork.currentValue

(['Gene3', 'Gene5'], 'Gene1')

In [29]:
gibbs.rvl[0].currentValue

(['Gene7'], 'Gene1')

In [30]:
sp.rand(5)

array([ 0.96826158,  0.31342418,  0.69232262,  0.87638915,  0.89460666])

In [31]:
kek2 = crv[0].getConditionalDistribution(hnrv.currentValue)

In [32]:
sp.rand(5)

array([ 0.08504421,  0.03905478,  0.16983042,  0.8781425 ,  0.09834683])

In [33]:
sp.rand(5)

array([ 0.42110763,  0.95788953,  0.53316528,  0.69187711,  0.31551563])

In [34]:
kek2

array([ 0.02383282,  0.02156482,  0.02633934,  0.02156482,  0.02633934,
        0.02156482,  0.02156482,  0.02156482,  0.02156482,  0.02156482,
        0.02383282,  0.01951266,  0.02383282,  0.01951266,  0.01951266,
        0.01951266,  0.01951266,  0.01951266,  0.02383282,  0.02910947,
        0.02383282,  0.02383282,  0.02383282,  0.02383282,  0.02383282,
        0.02383282,  0.01951266,  0.01951266,  0.01951266,  0.01951266,
        0.01951266,  0.02383282,  0.02383282,  0.02383282,  0.02383282,
        0.02383282,  0.01951266,  0.01951266,  0.01951266,  0.01951266,
        0.01951266,  0.01951266,  0.01951266,  0.01951266,  0.01951266,
        0.01951266])

In [35]:
sp.rand(5)

array([ 0.68650093,  0.83462567,  0.01828828,  0.75014431,  0.98886109])

In [36]:
valInd = roulette(kek2)

In [37]:
valInd

33